In [22]:
import pandas as pd
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
import pickle


In [17]:
df = pd.read_csv('Housing.csv')

binary_columns = ['airconditioning', 'prefarea']
df[binary_columns] = df[binary_columns].replace({'yes': 1, 'no': 0})
df['furnishingstatus'] = df['furnishingstatus'].replace({'unfurnished': 0, 'semi-furnished': 1, 'furnished': 2})

C:\Users\Administrator\AppData\Local\Temp\ipykernel_4104\1150284888.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[binary_columns] = df[binary_columns].replace({'yes': 1, 'no': 0})
C:\Users\Administrator\AppData\Local\Temp\ipykernel_4104\1150284888.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['furnishingstatus'] = df['furnishingstatus'].replace({'unfurnished': 0, 'semi-furnished': 1, 'furnished': 2})


In [18]:
features = ['area', 'bathrooms', 'airconditioning', 'bedrooms', 'stories', 'parking', 'prefarea', 'furnishingstatus']
X = df[features]
y = df['price']

In [19]:
X,y

(     area  bathrooms  airconditioning  bedrooms  stories  parking  prefarea  \
 0    7420          2                1         4        3        2         1   
 1    8960          4                1         4        4        3         0   
 2    9960          2                0         3        2        2         1   
 3    7500          2                1         4        2        3         1   
 4    7420          1                1         4        2        2         0   
 ..    ...        ...              ...       ...      ...      ...       ...   
 540  3000          1                0         2        1        2         0   
 541  2400          1                0         3        1        0         0   
 542  3620          1                0         2        1        0         0   
 543  2910          1                0         3        1        0         0   
 544  3850          1                0         3        2        0         0   
 
      furnishingstatus  
 0           

In [11]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Feature scaling (Standardization: mean=0, std=1)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

In [12]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Initialize the Linear Regression model
model = LinearRegression()

# Train the model on the training data
model.fit(X_train, y_train)

# Predict on the validation set
y_val_pred = model.predict(X_val)

# Predict on the test set
y_test_pred = model.predict(X_test)

# Evaluate the model
val_mse = mean_squared_error(y_val, y_val_pred)
test_mse = mean_squared_error(y_test, y_test_pred)

val_r2 = r2_score(y_val, y_val_pred)
test_r2 = r2_score(y_test, y_test_pred)

# Print the results
print(f"Validation MSE: {val_mse}")
print(f"Validation R2 Score: {val_r2}")

print(f"Test MSE: {test_mse}")
print(f"Test R2 Score: {test_r2}")


Validation MSE: 1567212451189.4426
Validation R2 Score: 0.6590200755256925
Test MSE: 1619489264636.9622
Test R2 Score: 0.6512104787271864


In [14]:
from sklearn.linear_model import Ridge, Lasso

# Ridge or Lasso Regression
ridge_model = Ridge(alpha=1.0)
lasso_model = Lasso(alpha=1.0)

# Train Ridge model
ridge_model.fit(X_train, y_train)
ridge_val_pred = ridge_model.predict(X_val)
ridge_test_pred = ridge_model.predict(X_test)

# Train Lasso model
lasso_model.fit(X_train, y_train)
lasso_val_pred = lasso_model.predict(X_val)
lasso_test_pred = lasso_model.predict(X_test)


In [15]:
# Evaluate Ridge model
ridge_val_mse = mean_squared_error(y_val, ridge_val_pred)
ridge_test_mse = mean_squared_error(y_test, ridge_test_pred)

ridge_val_r2 = r2_score(y_val, ridge_val_pred)
ridge_test_r2 = r2_score(y_test, ridge_test_pred)

# Evaluate Lasso model
lasso_val_mse = mean_squared_error(y_val, lasso_val_pred)
lasso_test_mse = mean_squared_error(y_test, lasso_test_pred)

lasso_val_r2 = r2_score(y_val, lasso_val_pred)
lasso_test_r2 = r2_score(y_test, lasso_test_pred)

# Print the results for Ridge
print(f"Ridge - Validation MSE: {ridge_val_mse}")
print(f"Ridge - Validation R2 Score: {ridge_val_r2}")
print(f"Ridge - Test MSE: {ridge_test_mse}")
print(f"Ridge - Test R2 Score: {ridge_test_r2}")

# Print the results for Lasso
print(f"Lasso - Validation MSE: {lasso_val_mse}")
print(f"Lasso - Validation R2 Score: {lasso_val_r2}")
print(f"Lasso - Test MSE: {lasso_test_mse}")
print(f"Lasso - Test R2 Score: {lasso_test_r2}")

Ridge - Validation MSE: 1568851087556.886
Ridge - Validation R2 Score: 0.6586635558308754
Ridge - Test MSE: 1620476504084.2915
Ridge - Test R2 Score: 0.6509978568952702
Lasso - Validation MSE: 1567214055549.7646
Lasso - Validation R2 Score: 0.6590197264634701
Lasso - Test MSE: 1619490302044.7207
Lasso - Test R2 Score: 0.6512102553006016


In [21]:
import numpy as np
import pandas as pd

# Convert 'yes'/'no' to binary (0 and 1)
df['mainroad'] = df['mainroad'].map({'yes': 1, 'no': 0})
df['guestroom'] = df['guestroom'].map({'yes': 1, 'no': 0})
df['basement'] = df['basement'].map({'yes': 1, 'no': 0})
df['hotwaterheating'] = df['hotwaterheating'].map({'yes': 1, 'no': 0})
df['airconditioning'] = df['airconditioning'].map({'yes': 1, 'no': 0})
df['prefarea'] = df['prefarea'].map({'yes': 1, 'no': 0})

# Convert 'furnishingstatus' to numeric (unfurnished=0, semi-furnished=1, furnished=2)
df['furnishingstatus'] = df['furnishingstatus'].map({'unfurnished': 0, 'semi-furnished': 1, 'furnished': 2})


# Create new interaction features
df['area_bedrooms'] = df['area'] * df['bedrooms']
df['area_bathrooms'] = df['area'] * df['bathrooms']
df['bed_bath'] = df['bedrooms'] * df['bathrooms']
df['guest_ac'] = df['guestroom'] * df['airconditioning']

# Split the data again
X = df[['area', 'bedrooms', 'bathrooms', 'stories', 'mainroad', 'guestroom', 
        'airconditioning', 'parking', 'prefarea', 'furnishingstatus', 
        'area_bedrooms', 'area_bathrooms', 'bed_bath', 'guest_ac']]
y = df['price']

# Train/Validation/Test split
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Feature scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)


C:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\extmath.py:1137: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
C:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\extmath.py:1142: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
C:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\extmath.py:1162: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count
